**Table of contents**<a id='toc0_'></a>    
- [简介](#toc1_)    
- [安装](#toc2_)    
- [使用](#toc3_)    
  - [基本使用](#toc3_1_)    
  - [文本补全](#toc3_2_)    
    - [简单对话](#toc3_2_1_)    
    - [返回`json`格式](#toc3_2_2_)    
      - [返回`pydantic`模型](#toc3_2_2_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[简介](#toc0_)

[openai-python](https://github.com/openai/openai-python)是`OpenAI`的官方`Python`库.

# <a id='toc2_'></a>[安装](#toc0_)

```bash
pip install openai
```

# <a id='toc3_'></a>[使用](#toc0_)

## <a id='toc3_1_'></a>[基本使用](#toc0_)

首先要创建一个客户端对象.

In [ ]:
import json

from openai import OpenAI

open_client = OpenAI(
    base_url="https://api.juheai.top/v1/",
    api_key='sk-RLbK5xgSbH1YsoYW8fA55c5b03864108930316F7178bF970'
)


## <a id='toc3_2_'></a>[文本补全](#toc0_)

### <a id='toc3_2_1_'></a>[简单对话](#toc0_)

In [ ]:
response = open_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": f"""你好"""
        }
    ]
)
result_text = response.choices[0].message.content.strip()
print(result_text)

### <a id='toc3_2_2_'></a>[返回`json`格式](#toc0_)

In [ ]:
response = open_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": f"""随机生成以下json格式的数据: 
        {{
            "name": "张三",
            "age": 18,
        }}
"""
        }
    ],
    response_format={"type": "json_object"}
)
# print(response)
result_text = response.choices[0].message.content.strip()
print(result_text)

#### <a id='toc3_2_2_1_'></a>[返回`pydantic`模型](#toc0_)

In [ ]:
from pydantic import BaseModel
from typing import TypeVar, Type

# 定义泛型类型变量
T = TypeVar('T', bound=BaseModel)


def generate_instance_from_prompt(open_client: OpenAI, prompt: str, model: Type[T]) -> T:
    """
    调用 OpenAI API，根据提示词生成符合指定 Pydantic 类型的实例.

    :param open_client: OpenAI 客户端对象
    :param prompt: 提示词
    :param model: Pydantic 泛型类型
    :return: Pydantic 类型的实例
    """
    # 调用 OpenAI API 生成 JSON 数据
    response = open_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"""{prompt},返回的数据需要符合以下 json schema: {json.dumps(model.model_json_schema(), indent=4)}
"""
            }
        ],
        response_format={"type": "json_object"}
    )
    # print(response)
    # 提取生成的 JSON 数据
    result_text = response.choices[0].message.content.strip()

    # 解析 JSON 数据为 Pydantic 实例
    data_dict = json.loads(result_text)
    instance = model(**data_dict)
    return instance


class Person(BaseModel):
    name: str
    age: int
    
instance = generate_instance_from_prompt(open_client, "随机生成一个人的信息", Person)

print(instance)

## 图像生成

## 语音转文本(Speech-to-Text)

In [ ]:
with open("audio.mp3", "rb") as audio_file:
    transcription = open_client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file
    )
    print(transcription.text)